## Problema del viajante

Resolver el problema del viajante utilizando algoritmos genéticos. 

- ¿Es posible aproximarse al resultado?

## Codificación

- Definir como se codificará el problema

## Adaptación

- Definir como se calculará la adaptación de un individuo
- ¿Existen estados que deben ser penalizados?


In [1]:
import numpy
import matplotlib
import matplotlib.pyplot as plt
import itertools
import json
import random


### Carga de datos Json

In [2]:
with open('distances.json', 'r') as file:
    distances = json.loads(file.read())

all_cities = list(distances.keys())
all_cities

['SR', 'EN', 'ES', 'JP', 'VA', 'LM', 'SDLR', 'RC', 'MC', 'GA', 'RA']

### Crear combinaciones

In [3]:
total_combinations = []
random_combinations = []
for combination in itertools.permutations(all_cities):
# Apendar primer ciudad al final de cada combinación para cumplir con el incio = final
    comb_list = list(combination)
    comb_list.append(combination[0])
    combination = tuple(comb_list)
    total_combinations.append(combination)

# Aumentar range para mas muestras en la población
for x in range(10):
    choice = random.choice(total_combinations)
    random_combinations.append(choice)
    total_combinations.remove(choice)
    
    
random_combinations

[('SDLR', 'RC', 'RA', 'LM', 'MC', 'VA', 'SR', 'JP', 'GA', 'ES', 'EN', 'SDLR'),
 ('RA', 'VA', 'SR', 'RC', 'ES', 'EN', 'GA', 'MC', 'LM', 'JP', 'SDLR', 'RA'),
 ('ES', 'GA', 'LM', 'EN', 'SDLR', 'RC', 'JP', 'SR', 'VA', 'MC', 'RA', 'ES'),
 ('RA', 'MC', 'SR', 'ES', 'RC', 'JP', 'SDLR', 'EN', 'VA', 'LM', 'GA', 'RA'),
 ('SR', 'ES', 'GA', 'RA', 'LM', 'VA', 'MC', 'JP', 'EN', 'SDLR', 'RC', 'SR'),
 ('SR', 'RC', 'JP', 'GA', 'EN', 'SDLR', 'ES', 'VA', 'RA', 'LM', 'MC', 'SR'),
 ('VA', 'MC', 'RA', 'EN', 'LM', 'ES', 'GA', 'JP', 'SR', 'SDLR', 'RC', 'VA'),
 ('VA', 'LM', 'JP', 'SR', 'ES', 'MC', 'SDLR', 'RC', 'RA', 'GA', 'EN', 'VA'),
 ('GA', 'ES', 'VA', 'SDLR', 'MC', 'LM', 'SR', 'JP', 'RC', 'EN', 'RA', 'GA'),
 ('GA', 'JP', 'LM', 'RC', 'MC', 'VA', 'SR', 'EN', 'ES', 'SDLR', 'RA', 'GA')]

### Calcular distancia por combinacion

In [4]:
def calculate_distance(combination: list) -> int:
    """Busco la distancia de la combinacion en el json"""
    distance = 0
    start = combination[0]
    combination_list = list(combination)
    combination_list.pop(0)
    combination = tuple(combination_list)
    
    for city in combination:
        distance += distances[start][city]
        start = city
        
    return distance

In [5]:
# Agrego las distancias de cada combinación random a una lista
metrics = []
for c in random_combinations:
    distance = metrics.append(calculate_distance(c))
metrics

[779.9,
 774.0999999999998,
 938.8000000000001,
 960.7,
 923.5,
 904.1,
 857.5999999999999,
 940.7,
 925.5,
 720.7]

## Función de Adaptación

In [6]:
def adaptation_function(population: list) -> list:
    fitness = []
    for individual in population:
        fitness.append(-individual)
    return fitness

adapted_population = adaptation_function(metrics)
adapted_population

[-779.9,
 -774.0999999999998,
 -938.8000000000001,
 -960.7,
 -923.5,
 -904.1,
 -857.5999999999999,
 -940.7,
 -925.5,
 -720.7]

## Selección

### Selección por torneo

In [7]:
def tournament_selection(adapted_population: list, random_combinations: list) -> list:
    """Esta funcion tiene que hacer el torneo entre 2 valores"""
    parents = []
    fitness = adapted_population
    adapted_c = adapted_population
    while len(adapted_population) > 0:
        selection1 = random.choice(adapted_c)
        selection2 = random.choice(adapted_c)
        
        
        if selection1 == selection2:
            continue
            
        if selection1 > selection2:
            # Obtiene el indice de la tupla de valores negativos(adaptación) para encontrar individuos "ganadores" de la población random
            parents.append(random_combinations[fitness.index(selection1)])
        else:
            parents.append(random_combinations[fitness.index(selection2)])
            
        adapted_c.remove(selection1)
        adapted_c.remove(selection2)

    return parents

parents = tournament_selection(adapted_population, random_combinations)
print(parents)

[('VA', 'MC', 'RA', 'EN', 'LM', 'ES', 'GA', 'JP', 'SR', 'SDLR', 'RC', 'VA'), ('SR', 'ES', 'GA', 'RA', 'LM', 'VA', 'MC', 'JP', 'EN', 'SDLR', 'RC', 'SR'), ('SDLR', 'RC', 'RA', 'LM', 'MC', 'VA', 'SR', 'JP', 'GA', 'ES', 'EN', 'SDLR'), ('RA', 'VA', 'SR', 'RC', 'ES', 'EN', 'GA', 'MC', 'LM', 'JP', 'SDLR', 'RA'), ('RA', 'VA', 'SR', 'RC', 'ES', 'EN', 'GA', 'MC', 'LM', 'JP', 'SDLR', 'RA')]


## Cruza

In [8]:
def uniform_cross(parents: list) -> list:
    """La cruza uniforme selecciona bits(ciudades) aleatorios del padre 1 y padre 2 creando 2 hijos y no se tienen que repetir ciudades"""
    silbings = []
    parents_copy = parents
    # Selecciono los 2 primeros padres y tiene que ser mayor a 1 para que puedan ser padres pares
    while len(parents_copy) > 1:
        
        parent1 = random.choice(parents_copy)
        parents_copy.remove(parent1)
        parent2 = random.choice(parents_copy)
        parents_copy.remove(parent2)
    
        child1 = list()
        child2 = list()
        
        for city1, city2 in zip(parent1, parent2):
            # Probabilidad 0,5 de cambiar
            change = round(random.uniform(0, 1), 2)
            if change <= 0.5 and city1 not in child1 and city2 not in child2:
                child1.append(city1)
                child2.append(city2)
            else:
                if city2 not in child1:
                    child1.append(city2)
                      
                if city1 not in child2:
                    child2.append(city1)
                    
        for city in parent1:
            if city not in child1:
                child1.insert(len(child1) - 2, city)
            if city not in child2:
                child2.insert(len(child2) - 2, city)
                
        # Correción para que el último valor 
        if child1[0] != child1[len(child1) -1]:
            child1[len(child1) -1] = child1[0]
        if child2[0] != child2[len(child2) -1]:
            child2[len(child2) -1] = child1[2]
            
        print(f"Hijo1 {child1} \n\nHijo2 {child2}\n\nPadre1{parent1}\n\nPadre2{parent2}\n\n")
        
        silbings.append(child1)
        silbings.append(child2)
        
    return silbings
        
silbings = uniform_cross(parents)
silbings
    

Hijo1 ['SR', 'MC', 'GA', 'EN', 'LM', 'ES', 'JP', 'SDLR', 'RA', 'RC', 'SR'] 

Hijo2 ['VA', 'ES', 'RA', 'LM', 'MC', 'JP', 'EN', 'SDLR', 'GA', 'RC', 'GA']

Padre1('SR', 'ES', 'GA', 'RA', 'LM', 'VA', 'MC', 'JP', 'EN', 'SDLR', 'RC', 'SR')

Padre2('VA', 'MC', 'RA', 'EN', 'LM', 'ES', 'GA', 'JP', 'SR', 'SDLR', 'RC', 'VA')


Hijo1 ['RA', 'VA', 'SR', 'RC', 'ES', 'EN', 'GA', 'MC', 'LM', 'JP', 'RA'] 

Hijo2 ['RA', 'VA', 'SR', 'RC', 'ES', 'EN', 'GA', 'MC', 'LM', 'JP', 'SR']

Padre1('RA', 'VA', 'SR', 'RC', 'ES', 'EN', 'GA', 'MC', 'LM', 'JP', 'SDLR', 'RA')

Padre2('RA', 'VA', 'SR', 'RC', 'ES', 'EN', 'GA', 'MC', 'LM', 'JP', 'SDLR', 'RA')




[['SR', 'MC', 'GA', 'EN', 'LM', 'ES', 'JP', 'SDLR', 'RA', 'RC', 'SR'],
 ['VA', 'ES', 'RA', 'LM', 'MC', 'JP', 'EN', 'SDLR', 'GA', 'RC', 'GA'],
 ['RA', 'VA', 'SR', 'RC', 'ES', 'EN', 'GA', 'MC', 'LM', 'JP', 'RA'],
 ['RA', 'VA', 'SR', 'RC', 'ES', 'EN', 'GA', 'MC', 'LM', 'JP', 'SR']]

## Mutación

# Algoritmo